1)Importar librerías y definir ruta

In [45]:
from pathlib import Path ##importa la clase Path del módulo pathlib, que permite trabajar con rutas de archivos de forma más segura 
import pandas as pd
import numpy as np

DATA_DIR = Path(r"d:\Users\usuario\Desktop\csv")


2)Cargar las tablas: En esta etapa se leen los cuatro archivos correspondientes a información de jugadores.  
Se verifica la cantidad de registros y la estructura general de las tablas para confirmar que los archivos se hayan cargado correctamente.


In [46]:
player  = pd.read_csv(DATA_DIR / "player.csv", sep=",", encoding="utf-8", low_memory=False)
common  = pd.read_csv(DATA_DIR / "common_player_info.csv", sep=",", encoding="utf-8", low_memory=False)
draft   = pd.read_csv(DATA_DIR / "draft_history.csv", sep=",", encoding="utf-8", low_memory=False)
combine = pd.read_csv(DATA_DIR / "draft_combine_stats.csv", sep=",", encoding="utf-8", low_memory=False)


 3)Inspección inicial: Confirmamos que los archivos se cargaron bien, qué dimensiones tienen y cómo se llaman sus columnas

In [47]:
print("Shapes (filas, columnas):")
print("player :", player.shape)
print("common :", common.shape)
print("draft  :", draft.shape)
print("combine:", combine.shape)

print("\nPLAYER cols:", player.columns.tolist())
print("COMMON  cols:", common.columns.tolist()[:15], "...")
print("DRAFT   cols:", draft.columns.tolist()[:15], "...")
print("COMBINE cols:", combine.columns.tolist()[:15], "...")


Shapes (filas, columnas):
player : (4831, 5)
common : (4171, 33)
draft  : (7990, 14)
combine: (1202, 47)

PLAYER cols: ['id', 'full_name', 'first_name', 'last_name', 'is_active']
COMMON  cols: ['person_id', 'first_name', 'last_name', 'display_first_last', 'display_last_comma_first', 'display_fi_last', 'player_slug', 'birthdate', 'school', 'country', 'last_affiliation', 'height', 'weight', 'season_exp', 'jersey'] ...
DRAFT   cols: ['person_id', 'player_name', 'season', 'round_number', 'round_pick', 'overall_pick', 'draft_type', 'team_id', 'team_city', 'team_name', 'team_abbreviation', 'organization', 'organization_type', 'player_profile_flag'] ...
COMBINE cols: ['season', 'player_id', 'first_name', 'last_name', 'player_name', 'position', 'height_wo_shoes', 'height_wo_shoes_ft_in', 'height_w_shoes', 'height_w_shoes_ft_in', 'weight', 'wingspan', 'wingspan_ft_in', 'standing_reach', 'standing_reach_ft_in'] ...


4)Normalizar nombres (snake_case) + unificar clave player_id
deja tus tablas listas para unificarlas más adelante:
Todos los nombres de columnas tienen el mismo estilo (snake_case).
Todas las tablas usan la misma columna clave (player_id).
Los tipos de datos son consistentes para poder hacer merge sin errores.

In [48]:

#Normalizar nombres de columnas

def snake_cols(df):#Limpia los nombres de columnas y los pasa a snake_case
    """
    Limpia y homogeneiza los nombres de las columnas de un DataFrame.
    Convierte a minúsculas, elimina espacios y reemplaza símbolos por '_'.
    """
    df = df.copy()
    df.columns = (df.columns
                    .str.strip()          # elimina espacios al inicio/final
                    .str.lower()          # pasa todo a minúsculas
                    .str.replace(" ", "_")
                    .str.replace("-", "_")
                    .str.replace("/", "_"))
    return df

# Auditoría rápida

def audit(df, name, head=3):##Muestra resumen del DataFrame (estructura, tipos, nulos, ejemplos)
    """
    Muestra información básica de un DataFrame:
    - Dimensiones
    - Tipos de datos
    - Porcentaje de nulos (top 10)
    - Primeras filas
    """
    print(f"\n=== AUDITORÍA: {name.upper()} ===")
    print("Shape (filas, columnas):", df.shape)
    print("\nTipos de datos:\n", df.dtypes.head(12))
    print("\nNulos (% top10):\n", (df.isna().mean()*100)
          .sort_values(ascending=False)
          .head(10)
          .round(2))
    print("\nPrimeras filas:\n", df.head(head))

#  Normalización de nombres y clave unificada

player  = snake_cols(player).rename(columns={"id": "player_id"})##Aplica limpieza a las tablas y renombra la clave id → player_id
common  = snake_cols(common)
draft   = snake_cols(draft)
combine = snake_cols(combine)

# Unificar clave 'player_id' en todas las tablas
for df in (player, common, draft, combine):##Cambia person_id a player_id y estandariza tipo str
    if "person_id" in df.columns:
        df.rename(columns={"person_id": "player_id"}, inplace=True)
    df["player_id"] = df["player_id"].astype(str)


#  Auditorías rápidas: Revisa rápidamente cada tabla para verificar la limpieza

audit(player, "player (raw)")
audit(common, "common (raw)")
audit(draft, "draft (raw)")
audit(combine, "combine (raw)")



=== AUDITORÍA: PLAYER (RAW) ===
Shape (filas, columnas): (4831, 5)

Tipos de datos:
 player_id     object
full_name     object
first_name    object
last_name     object
is_active      int64
dtype: object

Nulos (% top10):
 first_name    0.12
player_id     0.00
full_name     0.00
last_name     0.00
is_active     0.00
dtype: float64

Primeras filas:
   player_id            full_name first_name     last_name  is_active
0     76001       Alaa Abdelnaby       Alaa     Abdelnaby          0
1     76002      Zaid Abdul-Aziz       Zaid    Abdul-Aziz          0
2     76003  Kareem Abdul-Jabbar     Kareem  Abdul-Jabbar          0

=== AUDITORÍA: COMMON (RAW) ===
Shape (filas, columnas): (4171, 33)

Tipos de datos:
 player_id                   object
first_name                  object
last_name                   object
display_first_last          object
display_last_comma_first    object
display_fi_last             object
player_slug                 object
birthdate                   object
schoo

5)Limpiar player (catálogo base): Quitamos espacios, armamos full_name si hiciera falta, convertimos is_active a boolean, exigimos player_id, resolvemos duplicados por clave y dejamos lista la dimensión base de jugadores.

In [49]:
#Texto limpio: Recorre todas las columnas de tipo texto (object) y Convierte su contenido en string y elimina los espacios al principio y al final
for c in player.select_dtypes(include="object").columns:
    player[c] = player[c].astype(str).str.strip()

#Nombre preferido (si faltara), Comprueba si no existe la columna full_name. Si falta, la crea combinando first_name + last_name.
if "full_name" not in player.columns:
    player["full_name"] = (player.get("first_name","").fillna("") + " " +
                           player.get("last_name","").fillna("")).str.replace(r"\s+"," ", regex=True).str.strip()

#Tipos correctos, Convierte los valores de la columna is_active (que indican si el jugador sigue activo) a booleanos (True o False).
if "is_active" in player.columns:
    player["is_active"] = player["is_active"].map({1: True, 0: False, "1": True, "0": False}).astype("boolean")

#Reglas de faltantes críticos, Elimina las que no tengan valor en player_id (clave única del jugador). Muestra cuántas filas se eliminaron.
before = len(player)
player = player.dropna(subset=["player_id"])  # player_id no puede faltar
print("Filas eliminadas por player_id nulo:", before - len(player))

#Para visuales: Reemplaza los nombres vacíos a  'Unknown'
for col in ["full_name","first_name","last_name"]:
    if col in player.columns:
        player[col] = player[col].replace({"": np.nan}).fillna("Unknown")

#Duplicados por clave, Verifica si existen IDs duplicados. Elimina los duplicados y se queda con la primera ocurrencia.
##Cada jugador debe aparecer una sola vez en la tabla player.
print("Duplicados por player_id:", player.duplicated("player_id").sum())
player = player.drop_duplicates(subset=["player_id"], keep="first").reset_index(drop=True)

# Auditoría rápida, Muestra resumen del DataFrame
audit(player, "player (clean)")


Filas eliminadas por player_id nulo: 0
Duplicados por player_id: 0

=== AUDITORÍA: PLAYER (CLEAN) ===
Shape (filas, columnas): (4831, 5)

Tipos de datos:
 player_id      object
full_name      object
first_name     object
last_name      object
is_active     boolean
dtype: object

Nulos (% top10):
 player_id     0.0
full_name     0.0
first_name    0.0
last_name     0.0
is_active     0.0
dtype: float64

Primeras filas:
   player_id            full_name first_name     last_name  is_active
0     76001       Alaa Abdelnaby       Alaa     Abdelnaby      False
1     76002      Zaid Abdul-Aziz       Zaid    Abdul-Aziz      False
2     76003  Kareem Abdul-Jabbar     Kareem  Abdul-Jabbar      False


6)Limpieza de common_player_info: limpiar texto,parsear fecha de nacimiento,convertir altura a cm,convertir peso a kg,normalizar posición y crear position_group (G/F/C),estandarizar country/college,dejar 1 fila por player_id y auditar

In [50]:
#limpieza basica de texto
bio = common.copy()##crea una copia para no tocar el DataFrame original.

# Quitar espacios y normalizar strings
for c in bio.select_dtypes(include="object").columns:
    bio[c] = bio[c].astype(str).str.strip()

#Fecha de nacimiento (buscamos variantes)
bio["birth_date"] = pd.NaT
for cand in ["birthdate","birth_date","birth_day","date_of_birth"]:
    if cand in bio.columns:
        bio["birth_date"] = pd.to_datetime(bio[cand], errors="coerce")
        break

#Altura - cm 
import numpy as np

def height_to_cm(x):
    if pd.isna(x): 
        return np.nan
    s = str(x).strip()
    # Formato típico "6-8" (pies-pulgadas)
    if "-" in s:
        try:
            f, i = s.split("-", 1)
            return (int(f)*12 + int(i)) * 2.54
        except:
            return np.nan
    # Si ya viene como número (pulgadas o cm)
    try:
        v = float(s.replace(",", "."))
        # si es chico, asumimos pulgadas; si es grande, ya son cm
        return v if v > 100 else v * 2.54
    except:
        return np.nan

bio["height_cm"] = np.nan
for cand in ["height","player_height","height_inches","height_in","hgt"]:
    if cand in bio.columns:
        bio["height_cm"] = bio[cand].apply(height_to_cm)
        break

# Peso - kg 
def weight_to_kg(x):
    if pd.isna(x):
        return np.nan
    try:
        v = float(str(x).replace(",", "."))
        # si es grande (>150) asumimos libras; si es chico, ya está en kg
        return v/2.20462 if v > 150 else v
    except:
        return np.nan

bio["weight_kg"] = np.nan
for cand in ["weight","player_weight","weight_lbs","weight_lb","wgt"]:
    if cand in bio.columns:
        bio["weight_kg"] = bio[cand].apply(weight_to_kg)
        break

#Posición y grupo (G/F/C)
pos_col = next((c for c in ["position","player_position","pos"] if c in bio.columns), None)
if pos_col:
    bio["position"] = bio[pos_col].str.upper().str.replace(" ", "")
    bio["position_group"] = (bio["position"]
                             .str.replace("SG","G")
                             .str.replace("PG","G")
                             .str.replace("SF","F")
                             .str.replace("PF","F"))

# País / College 
rename_map = {"player_country":"country", "player_college":"college", "school":"college"}
for k, v in rename_map.items():
    if k in bio.columns and v not in bio.columns:
        bio.rename(columns={k: v}, inplace=True)

#Selección de columnas útiles + duplicados 
bio_keep = [c for c in [
    "player_id","birth_date","height_cm","weight_kg","position","position_group","country","college"
] if c in bio.columns]

bio = bio[bio_keep].copy()

#Normalizo texto vacío a NaN en campos descriptivos
for c in ["country","college","position","position_group"]:
    if c in bio.columns:
        bio[c] = bio[c].replace({"": np.nan})

#Un registro por jugador
bio = bio.drop_duplicates(subset=["player_id"]).reset_index(drop=True)

#Auditoría
audit(bio, "bio (clean)")



=== AUDITORÍA: BIO (CLEAN) ===
Shape (filas, columnas): (4171, 8)

Tipos de datos:
 player_id                 object
birth_date        datetime64[ns]
height_cm                float64
weight_kg                float64
position                  object
position_group            object
country                   object
college                   object
dtype: object

Nulos (% top10):
 weight_kg         2.40
height_cm         2.30
college           0.22
birth_date        0.00
player_id         0.00
position          0.00
position_group    0.00
country           0.00
dtype: float64

Primeras filas:
   player_id birth_date  height_cm   weight_kg       position position_group  \
0     76001 1968-06-24     208.28  108.862298        FORWARD        FORWARD   
1     76003 1947-04-16     218.44  102.058405         CENTER         CENTER   
2      1505 1974-11-03     198.12  106.594334  FORWARD-GUARD  FORWARD-GUARD   

  country         college  
0     USA            Duke  
1     USA            UCLA  


shape: (4171, 8)
birth_date → datetime64[ns] 
height_cm, weight_kg → float64 
Nulos bajos en altura/peso (~2.3–2.4%) y casi nada en el resto 
Columnas de texto (position, country, college) limpias 

7)Limpiar draft_history (un registro por jugador): renombrar a nombres estándar, castear numéricos y quedarnos con el primer draft por jugador.

In [51]:
# Copia
dh = draft.copy()

# Renombrar columnas a estándar
rename_draft = {
    "season":"draft_year",
    "round_number":"draft_round",
    "round_pick":"draft_pick_in_round",
    "overall_pick":"draft_overall_pick",
    "team_id":"draft_team_id",
    "team_abbreviation":"draft_team_abbr",
    "team_city":"draft_team_city"
}
for k,v in rename_draft.items():
    if k in dh.columns: dh.rename(columns={k:v}, inplace=True)

# Tipos correctos
for c in ["draft_year","draft_round","draft_pick_in_round","draft_overall_pick","draft_team_id"]:
    if c in dh.columns:
        dh[c] = pd.to_numeric(dh[c], errors="coerce")

# Faltantes críticos y 1 registro por jugador (PRIMER draft)
dh = dh.dropna(subset=["player_id"])
if "draft_year" in dh.columns:
    dh = dh.sort_values(["player_id","draft_year"], ascending=[True, True])
dh = dh.drop_duplicates(subset=["player_id"], keep="first").reset_index(drop=True)

# Selección de columnas útiles
dh_keep = [c for c in ["player_id","draft_year","draft_round","draft_pick_in_round",
                       "draft_overall_pick","draft_team_id","draft_team_abbr","draft_team_city"]
           if c in dh.columns]
dh = dh[dh_keep].copy()

# Auditoría
audit(dh, "draft (clean)")



=== AUDITORÍA: DRAFT (CLEAN) ===
Shape (filas, columnas): (7866, 8)

Tipos de datos:
 player_id              object
draft_year              int64
draft_round             int64
draft_pick_in_round     int64
draft_overall_pick      int64
draft_team_id           int64
draft_team_abbr        object
draft_team_city        object
dtype: object

Nulos (% top10):
 player_id              0.0
draft_year             0.0
draft_round            0.0
draft_pick_in_round    0.0
draft_overall_pick     0.0
draft_team_id          0.0
draft_team_abbr        0.0
draft_team_city        0.0
dtype: float64

Primeras filas:
   player_id  draft_year  draft_round  draft_pick_in_round  draft_overall_pick  \
0      1000        1996            2                   25                  54   
1      1001        1996            2                   26                  55   
2      1002        1996            2                   27                  56   

   draft_team_id draft_team_abbr draft_team_city  
0     161061276

chequeo 

In [52]:
print("filas:", len(dh))
print("player_id únicos:", dh["player_id"].nunique())
print("duplicados por player_id:", dh.duplicated("player_id").sum())

# (opcional) ver si quedan algunos con más de 1 fila
multi = dh["player_id"].value_counts()
print("jugadores con >1 fila:", (multi > 1).sum())


filas: 7866
player_id únicos: 7866
duplicados por player_id: 0
jugadores con >1 fila: 0


8)Limpiar draft_combine_stats (última medición por jugador)


In [53]:
# Copia
cb = combine.copy()

#detectar columna de año y ordenar: jugador ↑, año ↓ (más reciente primero)
year_col = next((c for c in ["season","season_year","draft_year","year"] if c in cb.columns), None)
if year_col:
    cb[year_col] = pd.to_numeric(cb[year_col], errors="coerce")
    cb = cb.sort_values(["player_id", year_col], ascending=[True, False])

#un registro por jugador
cb = cb.drop_duplicates(subset=["player_id"], keep="first").reset_index(drop=True)

#prefijo para no chocar nombres en el merge
cb = cb.rename(columns={c: (f"combine_{c}" if c != "player_id" else c) for c in cb.columns})

audit(cb, "combine (clean)")



=== AUDITORÍA: COMBINE (CLEAN) ===
Shape (filas, columnas): (1176, 47)

Tipos de datos:
 combine_season                     int64
player_id                         object
combine_first_name                object
combine_last_name                 object
combine_player_name               object
combine_position                  object
combine_height_wo_shoes          float64
combine_height_wo_shoes_ft_in     object
combine_height_w_shoes           float64
combine_height_w_shoes_ft_in      object
combine_weight                   float64
combine_wingspan                 float64
dtype: object

Nulos (% top10):
 combine_off_drib_college_break_right    97.62
combine_off_drib_college_top_key        97.62
combine_spot_fifteen_corner_left        94.22
combine_spot_fifteen_break_right        94.05
combine_spot_fifteen_break_left         94.05
combine_spot_fifteen_top_key            94.05
combine_spot_fifteen_corner_right       94.05
combine_off_drib_college_break_left     91.24
combine_on_move_c

In [54]:
# % de nulos por columna
null_pct = cb.isna().mean()

# Umbral: me quedo con columnas con <= 90% nulos
cols_by_threshold = [c for c in cb.columns if null_pct[c] <= 0.90]

# Métricas físicas que suelen interesar (si existen)
prefer = [
    "player_id", "combine_season",
    "combine_height_wo_shoes", "combine_weight", "combine_wingspan",
    "combine_body_fat", "combine_hand_length", "combine_hand_width",
    "combine_bench_press", "combine_lane_agility", "combine_three_quarter_sprint",
    "combine_shuttle_run", "combine_max_vertical", "combine_standing_vertical"
]

# Me quedo con las 'prefer' que existan + las que pasen el umbral
keep = [c for c in prefer if c in cb.columns] + [c for c in cols_by_threshold if c not in prefer]
cb = cb[keep].copy()

audit(cb, "combine (post-selección)")



=== AUDITORÍA: COMBINE (POST-SELECCIÓN) ===
Shape (filas, columnas): (1176, 38)

Tipos de datos:
 player_id                        object
combine_season                    int64
combine_height_wo_shoes         float64
combine_weight                  float64
combine_wingspan                float64
combine_hand_length             float64
combine_hand_width              float64
combine_bench_press             float64
combine_three_quarter_sprint    float64
combine_first_name               object
combine_last_name                object
combine_player_name              object
dtype: object

Nulos (% top10):
 combine_on_move_fifteen                 88.44
combine_off_drib_fifteen_break_left     86.90
combine_off_drib_fifteen_break_right    86.90
combine_off_drib_fifteen_top_key        86.90
combine_spot_college_break_right        86.48
combine_spot_college_top_key            86.48
combine_spot_college_corner_right       86.48
combine_spot_college_break_left         86.48
combine_spot_nba_cor

9)Consolidar Dim_Jugador y validacion final: Corre esto para unir todo y chequear calidad:
( SIENDO QC: CONTROL DE CALIDAD, VERIFICAMOS LA LIMPIEZA)

In [55]:
# Base de IDs para no perder jugadores
ids = (pd.Index(player["player_id"])
         .union(bio["player_id"])
         .union(dh["player_id"])
         .union(cb["player_id"]))
base = pd.DataFrame({"player_id": ids.astype(str)})

# Joins (left sobre la base)
dim_jugador = (base
    .merge(player, on="player_id", how="left")
    .merge(bio,    on="player_id", how="left")
    .merge(dh,     on="player_id", how="left")
    .merge(cb,     on="player_id", how="left")
)

# Seguridad: 1 fila por jugador
dim_jugador = dim_jugador.drop_duplicates(subset=["player_id"]).reset_index(drop=True)

# QC final
print("Filas:", len(dim_jugador), "| Jugadores únicos:", dim_jugador["player_id"].nunique())
print("Top nulos (%):\n", (dim_jugador.isna().mean()*100).sort_values(ascending=False).head(10).round(1))

if "height_cm" in dim_jugador.columns:
    print("Alturas fuera de rango (<150 o >240 cm):", dim_jugador.query("height_cm < 150 or height_cm > 240").shape[0])
if "weight_kg" in dim_jugador.columns:
    print("Pesos fuera de rango (<55 o >160 kg):", dim_jugador.query("weight_kg < 55 or weight_kg > 160").shape[0])

dim_jugador.head(3)


Filas: 9583 | Jugadores únicos: 9583
Top nulos (%):
 combine_on_move_fifteen                 98.6
combine_off_drib_fifteen_break_right    98.4
combine_off_drib_fifteen_break_left     98.4
combine_off_drib_fifteen_top_key        98.4
combine_spot_college_break_left         98.3
combine_spot_college_corner_right       98.3
combine_spot_college_break_right        98.3
combine_spot_college_top_key            98.3
combine_spot_nba_break_left             97.9
combine_spot_nba_break_right            97.9
dtype: float64
Alturas fuera de rango (<150 o >240 cm): 0
Pesos fuera de rango (<55 o >160 kg): 1


,player_id,full_name,first_name,last_name,is_active,birth_date,height_cm,weight_kg,position,position_group,...,combine_spot_college_corner_right,combine_spot_nba_corner_left,combine_spot_nba_break_left,combine_spot_nba_top_key,combine_spot_nba_break_right,combine_spot_nba_corner_right,combine_off_drib_fifteen_break_left,combine_off_drib_fifteen_top_key,combine_off_drib_fifteen_break_right,combine_on_move_fifteen
0,-1,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100,Tim Legler,Tim,Legler,False,1966-12-26,193.04,90.718582,GUARD,GUARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,Shandon Anderson,Shandon,Anderson,False,1973-12-31,198.12,97.522476,GUARD-FORWARD,GUARD-FORWARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
front = [c for c in [
    "player_id","full_name","first_name","last_name",
    "birth_date","country","college","position","position_group",
    "height_cm","weight_kg",
    "draft_year","draft_round","draft_pick_in_round","draft_overall_pick",
    "draft_team_id","draft_team_abbr","draft_team_city",
    "combine_season","combine_height_wo_shoes_cm","combine_wingspan_cm","combine_weight_kg"
] if c in dim_jugador.columns]

dim_jugador = dim_jugador[front + [c for c in dim_jugador.columns if c not in front]]


extra para que quede mas prolijo:Fila con player_id = '-1' debe venir de alguna tabla como “desconocido”. creo que es mejor eliminarla 

In [57]:
# quitar ids inválidos (negativos, vacíos)
dim_jugador = dim_jugador[dim_jugador["player_id"].str.strip().ne("-1")]
dim_jugador = dim_jugador[dim_jugador["player_id"].str.len() > 0].reset_index(drop=True)


Columnas del Combine con >95% nulos En el “Top nulos” hay métricas de tiro del combine con ~98% nulos. las podemos limpiarl para aligerar la tabla:

In [58]:
null_pct = dim_jugador.isna().mean()
cols_muy_vacias = [c for c, p in null_pct.items() if c.startswith("combine_") and p >= 0.95]

print("Voy a quitar:", len(cols_muy_vacias), "columnas casi vacías")
dim_jugador = dim_jugador.drop(columns=cols_muy_vacias)


Voy a quitar: 15 columnas casi vacías


Orden lindo de columnas: para que sean mas faciles de leer luego 

In [59]:
front = [c for c in [
    "player_id","full_name","first_name","last_name","is_active",
    "birth_date","country","college","position","position_group",
    "height_cm","weight_kg",
    "draft_year","draft_round","draft_pick_in_round","draft_overall_pick",
    "draft_team_id","draft_team_abbr","draft_team_city",
    "combine_season","combine_height_wo_shoes_cm","combine_wingspan_cm","combine_weight_kg"
] if c in dim_jugador.columns]

dim_jugador = dim_jugador[front + [c for c in dim_jugador.columns if c not in front]]


In [60]:
dim_jugador.head(5)



,player_id,full_name,first_name,last_name,is_active,birth_date,country,college,position,position_group,...,combine_height_wo_shoes_ft_in,combine_height_w_shoes,combine_height_w_shoes_ft_in,combine_wingspan_ft_in,combine_standing_reach,combine_standing_reach_ft_in,combine_body_fat_pct,combine_standing_vertical_leap,combine_max_vertical_leap,combine_lane_agility_time
0,100,Tim Legler,Tim,Legler,False,1966-12-26,USA,La Salle,GUARD,GUARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,Shandon Anderson,Shandon,Anderson,False,1973-12-31,USA,Georgia,GUARD-FORWARD,GUARD-FORWARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1002,Reggie Geary,Reggie,Geary,False,1973-08-31,USA,Arizona,GUARD,GUARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100263,Bill Laimbeer,Bill,Laimbeer,False,1900-01-01,USA,nan,CENTER,CENTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
dim_jugador.to_csv("dim_jugador.csv", index=False, encoding="utf-8")
print("Dim_Jugador exportada:", dim_jugador.shape)


Dim_Jugador exportada: (9582, 41)


Resumen ETL Dim Jugador

Se consolidaron player, common_player_info, draft_history y draft_combine_stats.
Se normalizaron nombres y claves (player_id), limpiando texto, duplicados y tipos de datos.
Se convirtieron alturas a cm, pesos a kg y fechas de nacimiento a formato datetime.
Se generó una sola fila por jugador (primer draft y última medición del combine).
Se eliminaron columnas con más del 95 % de nulos y registros inválidos (player_id = -1).
Resultado: tabla dim_jugador con información biográfica, deportiva y física lista para análisis o modelado.

CONTINUAMOS CON LA CONSOLIDACION DE DIM ESTADIO 

1)Cargar y auditar bases

In [62]:
game_info    = pd.read_csv(DATA_DIR / "game_info.csv", sep=",", encoding="utf-8", low_memory=False)
team_details = pd.read_csv(DATA_DIR / "team_details.csv", sep=",", encoding="utf-8", low_memory=False)

audit(game_info,    "game_info (raw)")       # solo game_id, game_date, attendance, game_time
audit(team_details, "team_details (raw)")    # donde está arena/city/arenacapacity




=== AUDITORÍA: GAME_INFO (RAW) ===
Shape (filas, columnas): (58053, 4)

Tipos de datos:
 game_id         int64
game_date      object
attendance    float64
game_time      object
dtype: object

Nulos (% top10):
 game_time     48.42
attendance     9.27
game_date      0.00
game_id        0.00
dtype: float64

Primeras filas:
     game_id            game_date  attendance game_time
0  24600001  1946-11-01 00:00:00         NaN       NaN
1  24600003  1946-11-02 00:00:00         NaN       NaN
2  24600002  1946-11-02 00:00:00         NaN       NaN

=== AUDITORÍA: TEAM_DETAILS (RAW) ===
Shape (filas, columnas): (25, 14)

Tipos de datos:
 team_id                 int64
abbreviation           object
nickname               object
yearfounded           float64
city                   object
arena                  object
arenacapacity         float64
owner                  object
generalmanager         object
headcoach              object
dleagueaffiliation     object
facebook               object
dtype

2)Limpiar y estandarizar team_details

In [63]:
td = snake_cols(team_details).copy()  # team_id, abbreviation, nickname, yearfounded, city, arena, arenacapacity, ...

# Renombrar a estándar
td.rename(columns={
    "arena":          "arena_name",
    "city":           "arena_city",
    "arenacapacity":  "arena_capacity"
}, inplace=True)

# Tipos y texto
if "arena_capacity" in td.columns:
    td["arena_capacity"] = pd.to_numeric(td["arena_capacity"], errors="coerce").astype("Int64")

for c in ["arena_name", "arena_city"]:
    if c in td.columns:
        td[c] = td[c].astype(str).str.strip()

# Nos quedamos con columnas relevantes + clave de equipo para la tabla puente
cols_keep = [c for c in ["team_id", "arena_name", "arena_city", "arena_capacity"] if c in td.columns]
td = td[cols_keep].dropna(subset=["arena_name"]).drop_duplicates().reset_index(drop=True)

audit(td, "team_details (clean: arena)")



=== AUDITORÍA: TEAM_DETAILS (CLEAN: ARENA) ===
Shape (filas, columnas): (25, 4)

Tipos de datos:
 team_id            int64
arena_name        object
arena_city        object
arena_capacity     Int64
dtype: object

Nulos (% top10):
 arena_capacity    36.0
team_id            0.0
arena_name         0.0
arena_city         0.0
dtype: float64

Primeras filas:
       team_id                arena_name arena_city  arena_capacity
0  1610612737          State Farm Arena    Atlanta           18729
1  1610612741             United Center    Chicago           21711
2  1610612742  American Airlines Center     Dallas           19200


3)Construir Dim_Estadio (arenas únicas + stadium_id)

In [64]:
import hashlib

# Arenas únicas (nombre + ciudad + capacidad si existe)
cols_arena = [c for c in ["arena_name", "arena_city", "arena_capacity"] if c in td.columns]
arenas = td[cols_arena].drop_duplicates().reset_index(drop=True)

def make_stadium_id(row):
    # Usamos nombre + ciudad si hay; si no, solo nombre
    key = f"{str(row.get('arena_name','')).lower().strip()}|{str(row.get('arena_city','')).lower().strip()}"
    return hashlib.md5(key.encode("utf-8")).hexdigest()[:12]  # id corto legible

arenas["stadium_id"] = arenas.apply(make_stadium_id, axis=1)

# Orden “lindo”
front = [c for c in ["stadium_id", "arena_name", "arena_city", "arena_capacity"] if c in arenas.columns]
dim_estadio = arenas[front + [c for c in arenas.columns if c not in front]].copy()

# 1 fila por stadium_id
dim_estadio = dim_estadio.drop_duplicates(subset=["stadium_id"]).reset_index(drop=True)
audit(dim_estadio, "dim_estadio (final)")



=== AUDITORÍA: DIM_ESTADIO (FINAL) ===
Shape (filas, columnas): (24, 4)

Tipos de datos:
 stadium_id        object
arena_name        object
arena_city        object
arena_capacity     Int64
dtype: object

Nulos (% top10):
 arena_capacity    37.5
stadium_id         0.0
arena_name         0.0
arena_city         0.0
dtype: float64

Primeras filas:
      stadium_id                arena_name arena_city  arena_capacity
0  d4ebeecacf67          State Farm Arena    Atlanta           18729
1  6dfeeb41e477             United Center    Chicago           21711
2  6de4fb384c85  American Airlines Center     Dallas           19200


4)Tabla puente Equipo → Estadio (para joins/Hechos)

In [65]:
team_stadium = td.merge(dim_estadio, on=["arena_name","arena_city"], how="left")[["team_id","stadium_id"]].drop_duplicates()
audit(team_stadium, "team_stadium (puente)")



=== AUDITORÍA: TEAM_STADIUM (PUENTE) ===
Shape (filas, columnas): (25, 2)

Tipos de datos:
 team_id        int64
stadium_id    object
dtype: object

Nulos (% top10):
 team_id       0.0
stadium_id    0.0
dtype: float64

Primeras filas:
       team_id    stadium_id
0  1610612737  d4ebeecacf67
1  1610612741  6dfeeb41e477
2  1610612742  6de4fb384c85


Aclaracion:
dim_estadio: 24 estadios únicos (bien para 25 equipos: alguno comparte o falta city y quedó uno único por nombre).
arena_capacity: ~37.5% nulos, se considera normal, ya que varios no traen capacidad en team_details. Se puede dejar nulo.

 5)Exportá los resultados

In [66]:
dim_estadio.to_csv("dim_estadio.csv", index=False, encoding="utf-8")
team_stadium.to_csv("team_stadium.csv", index=False, encoding="utf-8")
print("Exportado: dim_estadio.csv y team_stadium.csv")


Exportado: dim_estadio.csv y team_stadium.csv


Resumen ETL Dim Estadio

Se consolidaron team_details y game_info.
Se normalizaron nombres y tipos (arena_capacity → Int64) y se limpiaron duplicados y espacios.
Se renombraron columnas (arena_name, arena_city, arena_capacity) y se eliminaron filas sin nombre de arena.
Se generó un stadium_id único mediante hash MD5 (nombre + ciudad).
Se creó la tabla puente team_stadium (team_id ↔ stadium_id).
Resultado: dim_estadio con arenas únicas y team_stadium para relacionar equipos y estadios.

QC rápido (Control de Calidad)

In [67]:
# 1) Stadium_id único en Dim
assert dim_estadio["stadium_id"].is_unique

# 2) Mapeo 1–N: cada team_id debe tener exactamente 1 stadium_id (con este dataset)
assert team_stadium.groupby("team_id")["stadium_id"].nunique().max() == 1

# 3) Equipos sin estadio mapeado
sin_estadio = team_stadium["stadium_id"].isna().sum()
print("Equipos sin estadio mapeado:", sin_estadio)


Equipos sin estadio mapeado: 0


CONTINUAMOS CON LA CONSOLIDACION DE DIM EQUIPO

1)Cargar fuentes y auditar: ver estructura, nulos y decidir qué usar.

In [68]:
team             = pd.read_csv(DATA_DIR / "team.csv",             encoding="utf-8", low_memory=False)
team_details     = pd.read_csv(DATA_DIR / "team_details.csv",     encoding="utf-8", low_memory=False)
team_history     = pd.read_csv(DATA_DIR / "team_history.csv",     encoding="utf-8", low_memory=False)
team_info_common = pd.read_csv(DATA_DIR / "team_info_common.csv", encoding="utf-8", low_memory=False)

audit(team, "team (raw)"); audit(team_details, "team_details (raw)")
audit(team_history, "team_history (raw)"); audit(team_info_common, "team_info_common (raw)")



=== AUDITORÍA: TEAM (RAW) ===
Shape (filas, columnas): (30, 7)

Tipos de datos:
 id                int64
full_name        object
abbreviation     object
nickname         object
city             object
state            object
year_founded    float64
dtype: object

Nulos (% top10):
 id              0.0
full_name       0.0
abbreviation    0.0
nickname        0.0
city            0.0
state           0.0
year_founded    0.0
dtype: float64

Primeras filas:
            id            full_name abbreviation   nickname       city  \
0  1610612737        Atlanta Hawks          ATL      Hawks    Atlanta   
1  1610612738       Boston Celtics          BOS    Celtics     Boston   
2  1610612739  Cleveland Cavaliers          CLE  Cavaliers  Cleveland   

           state  year_founded  
0        Atlanta        1949.0  
1  Massachusetts        1946.0  
2           Ohio        1970.0  

=== AUDITORÍA: TEAM_DETAILS (RAW) ===
Shape (filas, columnas): (25, 14)

Tipos de datos:
 team_id                 int6

Resultado: diagnóstico inicial (shape, dtypes, nulos) de cada tabla.

2)Verificar si team_info_common está vacío (documentar y excluir si corresponde)

In [69]:
tic_is_empty = (team_info_common.shape[0] == 0) or (team_info_common.dropna(how="all").shape[0] == 0)
print("team_info_common vacío →", tic_is_empty)


team_info_common vacío → True


“Se auditó team_info_common y no contiene registros válidos; se excluye del proceso. La información equivalente está en team, team_details y team_history.”

3)Normalizar nombres y texto: homogeneidad para merges y referenciar columnas sin errores.

In [70]:
team         = snake_cols(team).copy()
team_details = snake_cols(team_details).copy()
team_history = snake_cols(team_history).copy()
if not tic_is_empty: team_info_common = snake_cols(team_info_common).copy()

for df in [team, team_details, team_history] + ([] if tic_is_empty else [team_info_common]):
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip()


Paso 4)Transformaciones específicas (solo las tablas válidas)

Como team_info_common está vacío, se excluye.

4.1)TEAM

In [71]:
team.rename(columns={"id": "team_id", "full_name": "team_name"}, inplace=True)
if team["team_id"].dtype != "object":
    team["team_id"] = team["team_id"].astype(str).str.strip()

team = (
    team[[c for c in ["team_id","team_name","nickname","abbreviation",
                      "team_code","city","state","year_founded"] if c in team.columns]]
    .drop_duplicates(subset=["team_id"])
    .reset_index(drop=True)
)
audit(team, "team (clean)")



=== AUDITORÍA: TEAM (CLEAN) ===
Shape (filas, columnas): (30, 7)

Tipos de datos:
 team_id          object
team_name        object
nickname         object
abbreviation     object
city             object
state            object
year_founded    float64
dtype: object

Nulos (% top10):
 team_id         0.0
team_name       0.0
nickname        0.0
abbreviation    0.0
city            0.0
state           0.0
year_founded    0.0
dtype: float64

Primeras filas:
       team_id            team_name   nickname abbreviation       city  \
0  1610612737        Atlanta Hawks      Hawks          ATL    Atlanta   
1  1610612738       Boston Celtics    Celtics          BOS     Boston   
2  1610612739  Cleveland Cavaliers  Cavaliers          CLE  Cleveland   

           state  year_founded  
0        Atlanta        1949.0  
1  Massachusetts        1946.0  
2           Ohio        1970.0  


4.2 TEAM_HISTORY

In [72]:
if "id" in team_history.columns and "team_id" not in team_history.columns:
    team_history.rename(columns={"id":"team_id"}, inplace=True)
if team_history["team_id"].dtype != "object":
    team_history["team_id"] = team_history["team_id"].astype(str).str.strip()

for col in ["year_founded","year_active_till","min_year","max_year"]:
    if col in team_history.columns:
        team_history[col] = pd.to_numeric(team_history[col], errors="coerce").astype("Int64")

th = (
    team_history[[c for c in ["team_id","year_founded","year_active_till","min_year","max_year"]
                  if c in team_history.columns]]
    .drop_duplicates(subset=["team_id"])
    .reset_index(drop=True)
)
audit(th, "team_history (clean)")



=== AUDITORÍA: TEAM_HISTORY (CLEAN) ===
Shape (filas, columnas): (25, 3)

Tipos de datos:
 team_id             object
year_founded         Int64
year_active_till     Int64
dtype: object

Nulos (% top10):
 team_id             0.0
year_founded        0.0
year_active_till    0.0
dtype: float64

Primeras filas:
       team_id  year_founded  year_active_till
0  1610612737          1968              2019
1  1610612741          1966              2019
2  1610612742          1980              2019


 4.3 TEAM_DETAILS

In [73]:
team_details.rename(columns={
    "arena":"arena_name",
    "city":"arena_city",
    "arenacapacity":"arena_capacity",
    "id":"team_id"
}, inplace=True)

if team_details["team_id"].dtype != "object":
    team_details["team_id"] = team_details["team_id"].astype(str).str.strip()

if "arena_capacity" in team_details.columns:
    team_details["arena_capacity"] = pd.to_numeric(team_details["arena_capacity"], errors="coerce").astype("Int64")

td = team_details[[c for c in ["team_id","arena_name","arena_city","arena_capacity"] if c in team_details.columns]].copy()
null_pct_td = td.isna().mean()
td = td[[c for c in td.columns if null_pct_td.get(c, 0) <= 0.90]]

if "arena_name" in td.columns:
    td = td.dropna(subset=["arena_name"])
td = td.drop_duplicates(subset=["team_id"]).reset_index(drop=True)

audit(td, "team_details (clean)")



=== AUDITORÍA: TEAM_DETAILS (CLEAN) ===
Shape (filas, columnas): (25, 4)

Tipos de datos:
 team_id           object
arena_name        object
arena_city        object
arena_capacity     Int64
dtype: object

Nulos (% top10):
 arena_capacity    36.0
team_id            0.0
arena_name         0.0
arena_city         0.0
dtype: float64

Primeras filas:
       team_id                arena_name arena_city  arena_capacity
0  1610612737          State Farm Arena    Atlanta           18729
1  1610612741             United Center    Chicago           21711
2  1610612742  American Airlines Center     Dallas           19200


5)Validaciones finales

In [74]:
assert team["team_id"].is_unique
assert th["team_id"].is_unique
assert td["team_id"].is_unique
assert "arena_name" in td.columns and not td["arena_name"].isna().any()


6)Exportar

In [75]:
team.to_csv("dim_team.csv", index=False, encoding="utf-8")
th.to_csv("team_history_clean.csv", index=False, encoding="utf-8")
td.to_csv("team_details_clean.csv", index=False, encoding="utf-8")


Resumen ETL Dim Equipo:

Se consolidaron team, team_history y team_details.

team_info_common auditado pero excluido por vacío.

Se normalizaron nombres, tipos y claves (team_id).

Se eliminaron duplicados y columnas con >90% nulos.

Resultado: 3 DataFrames listos para EDA. 